# IBM Data Science Capstone Project - The Battle of Neighborhoods

# Looking for a property in Sydney

## Backgroud

A client came from Hong Kong is looking for an Sydney comforable home which is close to a train station within radius 20km from the Sydney central business district (CBD). To decide which location to look at, the clients wants to have a sense of community and neighbourhood of each possible train station. As a data analyst in the property company, I am asked to provide a neighbourhood analysis of all related train stations by using any online resources.

## Data

To solve the problem by using data science methodology, we use the Foursquare API to obtain a list of train stations within 20km radius from the Sydney CBD. We will then explore and analyse each train station by using the explore function to get the most common venue categories, and use this feature to group the train station into clusters. We will use the k-means clustering algorithm and examine the characteristics of each clusters. We will have a glance over the top 5 most common venues of each train stations. We will then use the folium library to visualize the train stations in the Sydney map and their emerging clusters. Finally, we will examine and provide a brief summary of each cluster.

## 1. Create a list of train stations within 20km radius of Sydney CBD

### 1.1 Download and import all the dependencies that we will need

In [1]:
import numpy as np # library to handle data in a vectorized mannerD
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 92kB 15.0MB/s eta 0:00:01
Libraries imported.


### 1.2 Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'T5F4IPH15CZ2MG15H2EDIVZIYZ2SD1YQESYECTRFARDYARPQ' # your Foursquare ID
CLIENT_SECRET = 'MEZMXAASYWY5YUDDL53ZMS4N5MMR3BEMUE3ARZKZDO3JEOCJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T5F4IPH15CZ2MG15H2EDIVZIYZ2SD1YQESYECTRFARDYARPQ
CLIENT_SECRET:MEZMXAASYWY5YUDDL53ZMS4N5MMR3BEMUE3ARZKZDO3JEOCJ


### 1.3 Get the latitude and longitude coordinates for Sydney CBD

In [3]:
syd_latitude = -33.8688
syd_longitude = 151.2093

### 1.4 Get all the train stations within 20km from Sydney CBD

In [4]:
radius=20000
LIMIT=500
train_categoryId='4bf58dd8d48988d129951735'

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET, 
    VERSION, 
    syd_latitude, 
    syd_longitude,
    train_categoryId,
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?&client_id=T5F4IPH15CZ2MG15H2EDIVZIYZ2SD1YQESYECTRFARDYARPQ&client_secret=MEZMXAASYWY5YUDDL53ZMS4N5MMR3BEMUE3ARZKZDO3JEOCJ&v=20180605&ll=-33.8688,151.2093&categoryId=4bf58dd8d48988d129951735&radius=20000&limit=500'

In [5]:
#create the get request URL
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d4abb89018cbb002c7eb31f'},
 'response': {'venues': [{'id': '4b058770f964a520539322e3',
    'name': 'Wynyard Station',
    'location': {'address': 'George St.',
     'crossStreet': 'Underground',
     'lat': -33.865676078654374,
     'lng': 151.20616275025094,
     'labeledLatLngs': [{'label': 'display',
       'lat': -33.865676078654374,
       'lng': 151.20616275025094}],
     'distance': 452,
     'postalCode': '2000',
     'cc': 'AU',
     'city': 'Sydney',
     'state': 'NSW',
     'country': 'Australia',
     'formattedAddress': ['George St. (Underground)',
      'Sydney NSW 2000',
      'Australia']},
    'categories': [{'id': '4bf58dd8d48988d129951735',
      'name': 'Train Station',
      'pluralName': 'Train Stations',
      'shortName': 'Train Station',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1565178761',
    'hasPerk': 

In [6]:
#Structure the json data into panda dataframe
venues = results['response']['venues']
#flatten JSON   
nearby_venues = json_normalize(venues)
nearby_venues.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4b058770f964a520539322e3,George St.,AU,Sydney,Australia,Underground,452,"[George St. (Underground), Sydney NSW 2000, Au...","[{'label': 'display', 'lat': -33.8656760786543...",-33.865676,151.206163,NaN,2000,NSW,Wynyard Station,v-1565178761,NaN
1,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,55adf7ff498e688326dce3ed,George St.,AU,Sydney,Australia,Underground at Druitt & Bathurst Sts,587,[George St. (Underground at Druitt & Bathurst ...,"[{'label': 'display', 'lat': -33.8736438265870...",-33.873644,151.206781,NaN,2000,NSW,Town Hall Station,v-1565178761,NaN
2,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,59be05e73149b92b3cd29e0e,Ormonde Parade,AU,Hurstville,Australia,NaN,14761,"[Ormonde Parade, Hurstville Grove NSW 2220, Au...","[{'label': 'display', 'lat': -33.9675542357079...",-33.967554,151.102658,Hurstville,2220,NSW,Hurstville Station,v-1565178761,NaN
3,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4b0631a4f964a520a2e922e3,Brown St.,AU,Ashfield,Australia,Station St.,8026,"[Brown St. (Station St.), Ashfield NSW 2131, A...","[{'label': 'display', 'lat': -33.8878545333653...",-33.887855,151.125538,NaN,2131,NSW,Ashfield Station,v-1565178761,NaN
4,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4c78a3d4bd346dcbdb0af3ef,George St.,AU,Sydney,Australia,btwn Druitt & Bathurst Sts.,626,"[George St. (btwn Druitt & Bathurst Sts.), Syd...","[{'label': 'display', 'lat': -33.8740506048304...",-33.874051,151.206850,NaN,2000,NSW,Town Hall Station (Main Concourse),v-1565178761,NaN


In [7]:
# filter columns, only keep the station name and it's latitude and longitude
filtered_columns = ['name', 'location.lat', 'location.lng']
station_list = nearby_venues[filtered_columns]
station_list.head()

,name,location.lat,location.lng
0,Wynyard Station,-33.865676,151.206163
1,Town Hall Station,-33.873644,151.206781
2,Hurstville Station,-33.967554,151.102658
3,Ashfield Station,-33.887855,151.125538
4,Town Hall Station (Main Concourse),-33.874051,151.206850


In [8]:
#clean the station list

station_list = station_list[~station_list['name'].str.contains('Concourse')]
station_list = station_list[~station_list['name'].str.contains('Platform')]
station_list = station_list[~station_list['name'].str.contains('Interchange')]
station_list.head()

,name,location.lat,location.lng
0,Wynyard Station,-33.865676,151.206163
1,Town Hall Station,-33.873644,151.206781
2,Hurstville Station,-33.967554,151.102658
3,Ashfield Station,-33.887855,151.125538
5,Clyde Station,-33.835988,151.016994


## 2. Explore venues within 500m from each train stations

### 2.1 Get the venues within 500m from each train station by using API request URL

In [9]:
#create a function to get venues of each train stations
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
#create a new dataframe called station_venues
station_venues = getNearbyVenues(names=station_list['name'],
                                   latitudes=station_list['location.lat'],
                                   longitudes=station_list['location.lng']
                                  )

Wynyard Station
Town Hall Station
Hurstville Station
Ashfield Station
Clyde Station
Bankstown Station
Auburn Station
Kings Cross Station
International Airport Station
Wolli Creek Station
Kogarah Station
Campsie Station
Chatswood Station
Strathfield Station
Parramatta Station
Milsons Point Station
Fairfield Station
Artarmon Station
Rhodes Station
Domestic Airport Station
Green Square Station
Burwood Station
St Peters Station
Edgecliff Station
Sydenham Station
Newtown Station
St Leonards Station
Redfern Station
Mascot Station
Gordon Station
Petersham Station
Central Station
Stanmore Station
Hornsby Station
Meadowbank Station
Dulwich Hill Station
Macquarie University Station
Rockdale Station


In [11]:
# check the size of the resulting dataframe
print(station_venues.shape)
station_venues.head()

(1507, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wynyard Station,-33.865676,151.206163,Regiment,-33.867043,151.206697,Café
1,Wynyard Station,-33.865676,151.206163,Made In Italy,-33.866514,151.205396,Italian Restaurant
2,Wynyard Station,-33.865676,151.206163,Mercado,-33.866901,151.207448,Spanish Restaurant
3,Wynyard Station,-33.865676,151.206163,Nook Urban Fresh Bar,-33.865767,151.204862,Café
4,Wynyard Station,-33.865676,151.206163,City Recital Hall Angel Place,-33.866898,151.207743,Concert Hall


### 2.2 Investigate how many venues were return for each train station

In [12]:
# check how many venues were returned for each train station
station_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Artarmon Station,19,19,19,19,19,19
Ashfield Station,36,36,36,36,36,36
Auburn Station,24,24,24,24,24,24
Bankstown Station,53,53,53,53,53,53
Burwood Station,46,46,46,46,46,46
Campsie Station,17,17,17,17,17,17
Central Station,84,84,84,84,84,84
Chatswood Station,43,43,43,43,43,43
Clyde Station,6,6,6,6,6,6


## 3. Analyze each train station

### 3.1 Create a list of venues for each train station

In [13]:
# one hot encoding
station_onehot = pd.get_dummies(station_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
station_onehot['Neighborhood'] = station_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [station_onehot.columns[-1]] + list(station_onehot.columns[:-1])
station_onehot = station_onehot[fixed_columns]

station_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Garden,Big Box Store,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Café,Candy Store,Cantonese Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Duty-free Shop,Egyptian Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hawaiian Restaurant,Health Food Store,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Iraqi Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Music Venue,Newsstand,Nightclub,Noodle House,Office,Other Nightlife,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Soccer Field,South American Restaurant,South Indian Restaurant,Souvlaki Shop,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Wynyard Station,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wynyard Station,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wynyard Station,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [14]:
print('There are {} uniques categories.'.format(len(station_venues['Venue Category'].unique())))

There are 200 uniques categories.


### 3.2 Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [15]:
station_grouped = station_onehot.groupby('Neighborhood').mean().reset_index()
station_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Garden,Big Box Store,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Café,Candy Store,Cantonese Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Duty-free Shop,Egyptian Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hawaiian Restaurant,Health Food Store,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Iraqi Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Music Venue,Newsstand,Nightclub,Noodle House,Office,Other Nightlife,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Soccer Field,South American Restaurant,South Indian Restaurant,Souvlaki Shop,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Artarmon Station,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.052632,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.210526,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.105263,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.0

In [16]:
# check data size
station_grouped.shape

(38, 201)

### 3.3 Print each train station along with the top 5 most common venues

In [17]:
num_top_venues = 5

for hood in station_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = station_grouped[station_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Artarmon Station----
                    venue  freq
0                    Café  0.21
1     Japanese Restaurant  0.11
2         Thai Restaurant  0.11
3                    Park  0.11
4  Furniture / Home Store  0.05


----Ashfield Station----
                 venue  freq
0  Dumpling Restaurant  0.08
1     Asian Restaurant  0.08
2    Electronics Store  0.06
3  Shanghai Restaurant  0.06
4          Supermarket  0.06


----Auburn Station----
                venue  freq
0                Café  0.12
1       Grocery Store  0.12
2              Bakery  0.08
3         Supermarket  0.08
4  Turkish Restaurant  0.08


----Bankstown Station----
                   venue  freq
0  Vietnamese Restaurant  0.19
1                   Café  0.09
2            Coffee Shop  0.04
3             Steakhouse  0.04
4                 Buffet  0.04


----Burwood Station----
                venue  freq
0                Café  0.13
1  Chinese Restaurant  0.09
2        Noodle House  0.07
3         Supermarket  0.07
4        

### 3.4 Put them into a pandas dataframe

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = station_grouped['Neighborhood']

for ind in np.arange(station_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(station_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Artarmon Station,Café,Park,Japanese Restaurant,Thai Restaurant,Sandwich Place,Furniture / Home Store,Sushi Restaurant,BBQ Joint,Asian Restaurant,Motel,Diner,Ramen Restaurant,Convenience Store,Falafel Restaurant,Food Court,Food & Drink Shop,Flea Market,Fish Market,Dim Sum Restaurant,Fast Food Restaurant
1,Ashfield Station,Dumpling Restaurant,Asian Restaurant,Shanghai Restaurant,Electronics Store,Japanese Restaurant,Supermarket,Chinese Restaurant,Coffee Shop,Korean Restaurant,Falafel Restaurant,Café,Department Store,Restaurant,Malay Restaurant,Bar,Liquor Store,Thai Restaurant,Polish Restaurant,Platform,Pharmacy
2,Auburn Station,Café,Grocery Store,Bakery,Turkish Restaurant,Supermarket,Tea Room,Thai Restaurant,Pakistani Restaurant,Lebanese Restaurant,Gym,Department Store,Fast Food Restaurant,Portuguese Restaurant,Persian Restaurant,Dessert Shop,Afghan Restaurant,Kebab Restaurant,Farmers Market,Falafel Restaurant,Event Space
3,Bankstown Station,Vietnamese Restaurant,Café,Buffet,Coffee Shop,Grocery Store,Middle Eastern Restaurant,Steakhouse,Chinese Restaurant,Sports Bar,Department Store,Sports Club,Bar,Supermarket,Record Shop,Portuguese Restaurant,Multiplex,Shopping Mall,Bus Station,Fast Food Restaurant,Sandwich Place
4,Burwood Station,Café,Chinese Restaurant,Noodle House,Supermarket,Department Store,Fast Food Restaurant,Coffee Shop,Sandwich Place,Cantonese Restaurant,Fried Chicken Joint,Middle Eastern Restaurant,Shopping Mall,Salad Place,Hookah Bar,Multiplex,Record Shop,Pub,Other Nightlife,Cosmetics Shop,Pharmacy


## 4. Cluster train stations

### 4.1 Run k-means to cluster the neighborhood into 5 clusters

In [20]:
# set number of clusters
kclusters = 5

station_grouped_clustering = station_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(station_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 2, 2, 2, 2, 1, 1, 0, 2], dtype=int32)

In [21]:
station_list

,name,location.lat,location.lng
0,Wynyard Station,-33.865676,151.206163
1,Town Hall Station,-33.873644,151.206781
2,Hurstville Station,-33.967554,151.102658
3,Ashfield Station,-33.887855,151.125538
5,Clyde Station,-33.835988,151.016994
7,Bankstown Station,-33.917855,151.034488
8,Auburn Station,-33.849409,151.032759
9,Kings Cross Station,-33.874495,151.222429
10,International Airport Station,-33.935120,151.166097
11,Wolli Creek Station,-33.928661,151.154010


In [22]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Artarmon Station,Café,Park,Japanese Restaurant,Thai Restaurant,Sandwich Place,Furniture / Home Store,Sushi Restaurant,BBQ Joint,Asian Restaurant,Motel,Diner,Ramen Restaurant,Convenience Store,Falafel Restaurant,Food Court,Food & Drink Shop,Flea Market,Fish Market,Dim Sum Restaurant,Fast Food Restaurant
1,Ashfield Station,Dumpling Restaurant,Asian Restaurant,Shanghai Restaurant,Electronics Store,Japanese Restaurant,Supermarket,Chinese Restaurant,Coffee Shop,Korean Restaurant,Falafel Restaurant,Café,Department Store,Restaurant,Malay Restaurant,Bar,Liquor Store,Thai Restaurant,Polish Restaurant,Platform,Pharmacy
2,Auburn Station,Café,Grocery Store,Bakery,Turkish Restaurant,Supermarket,Tea Room,Thai Restaurant,Pakistani Restaurant,Lebanese Restaurant,Gym,Department Store,Fast Food Restaurant,Portuguese Restaurant,Persian Restaurant,Dessert Shop,Afghan Restaurant,Kebab Restaurant,Farmers Market,Falafel Restaurant,Event Space
3,Bankstown Station,Vietnamese Restaurant,Café,Buffet,Coffee Shop,Grocery Store,Middle Eastern Restaurant,Steakhouse,Chinese Restaurant,Sports Bar,Department Store,Sports Club,Bar,Supermarket,Record Shop,Portuguese Restaurant,Multiplex,Shopping Mall,Bus Station,Fast Food Restaurant,Sandwich Place
4,Burwood Station,Café,Chinese Restaurant,Noodle House,Supermarket,Department Store,Fast Food Restaurant,Coffee Shop,Sandwich Place,Cantonese Restaurant,Fried Chicken Joint,Middle Eastern Restaurant,Shopping Mall,Salad Place,Hookah Bar,Multiplex,Record Shop,Pub,Other Nightlife,Cosmetics Shop,Pharmacy
5,Campsie Station,Malay Restaurant,Playground,Korean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Shopping Mall,Sushi Restaurant,Café,Liquor Store,Food & Drink Shop,Gym,Chinese Restaurant,Pharmacy,Department Store,Japanese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Fish Market,Dessert Shop,Farmers Market
6,Central Station,Café,Thai Restaurant,Coffee Shop,Japanese Restaurant,Hostel,Sandwich Place,Bar,Pub,Vietnamese Restaurant,Burger Joint,Australian Restaurant,Hotel,Chinese Restaurant,Asian Restaurant,Cocktail Bar,Breakfast Spot,Ramen Restaurant,Pool,Pizza Place,Speakeasy
7,Chatswood Station,Café,Coffee Shop,Food Court,Thai Restaurant,Japanese Restaurant,Gym,Chinese Restaurant,Ramen Restaurant,Chaat Place,Portuguese Restaurant,Burger Joint,Bubble Tea Shop,Tea Room,Shopping Mall,Dumpling Restaurant,Fried Chicken Joint,Performing Arts Venue,Market,Malay Restaurant,Sushi Restaurant
8,Clyde Station,Platform,Gym,Asian Restaurant,Rental Car Location,Train Station,Food Court,Food & Drink Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Egyptian Restaurant,Yoga Studio,Duty-free Shop,French Restaurant,Dumpling Restaurant,Donut Shop
9,Domestic Airport Station,Café,Fast Food Restaurant,Coffee Shop,Airport Lounge,Airport Terminal,Donut Shop,Juice Bar,Airport Service,Airport Gate,Bookstore,Bar,Tapas Restaurant,Bakery,Airport Food Court,Moving Target,Salad Place,Sandwich Place,Asian Restaurant,Rental Car Location,Train Station


In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

station_merged = station_list

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
station_merged = station_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='name')

station_merged.head() # check the last columns!

,name,location.lat,location.lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Wynyard Station,-33.865676,151.206163,1,Café,Coffee Shop,Bar,Speakeasy,Cocktail Bar,Sandwich Place,Italian Restaurant,Bakery,Restaurant,Hotel,French Restaurant,Clothing Store,Spanish Restaurant,Dessert Shop,Steakhouse,South Indian Restaurant,Electronics Store,Boutique,Shopping Mall,Malay Restaurant
1,Town Hall Station,-33.873644,151.206781,2,Café,Japanese Restaurant,Hotel,Coffee Shop,Bookstore,Cocktail Bar,Korean Restaurant,Gym,Thai Restaurant,Shopping Mall,Burger Joint,Record Shop,Speakeasy,Bar,Australian Restaurant,Tea Room,Hobby Shop,Hotel Bar,Ice Cream Shop,Malay Restaurant
2,Hurstville Station,-33.967554,151.102658,2,Chinese Restaurant,Fast Food Restaurant,Supermarket,Café,Bakery,Dumpling Restaurant,Vietnamese Restaurant,Coffee Shop,Japanese Restaurant,Bubble Tea Shop,Men's Store,Middle Eastern Restaurant,Souvlaki Shop,Accessories Store,Shopping Mall,Sandwich Place,Clothing Store,Electronics Store,Department Store,Bar
3,Ashfield Station,-33.887855,151.125538,2,Dumpling Restaurant,Asian Restaurant,Shanghai Restaurant,Electronics Store,Japanese Restaurant,Supermarket,Chinese Restaurant,Coffee Shop,Korean Restaurant,Falafel Restaurant,Café,Department Store,Restaurant,Malay Restaurant,Bar,Liquor Store,Thai Restaurant,Polish Restaurant,Platform,Pharmacy
5,Clyde Station,-33.835988,151.016994,0,Platform,Gym,Asian Restaurant,Rental Car Location,Train Station,Food Court,Food & Drink Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Egyptian Restaurant,Yoga Studio,Duty-free Shop,French Restaurant,Dumpling Restaurant,Donut Shop


### 4.2 Visualize the resulting clusters

In [24]:
# create map
map_clusters = folium.Map(location=[syd_latitude, syd_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(station_merged['location.lat'], station_merged['location.lng'], station_merged['name'], station_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

### 5.1 Cluster 1

In [25]:
station_merged.loc[station_merged['Cluster Labels'] == 0, station_merged.columns[[0] + list(range(4, station_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
5,Clyde Station,Platform,Gym,Asian Restaurant,Rental Car Location,Train Station,Food Court,Food & Drink Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Egyptian Restaurant,Yoga Studio,Duty-free Shop,French Restaurant,Dumpling Restaurant,Donut Shop


### 5.2 Cluster 2

In [26]:
station_merged.loc[station_merged['Cluster Labels'] == 1, station_merged.columns[[0] + list(range(4, station_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Wynyard Station,Café,Coffee Shop,Bar,Speakeasy,Cocktail Bar,Sandwich Place,Italian Restaurant,Bakery,Restaurant,Hotel,French Restaurant,Clothing Store,Spanish Restaurant,Dessert Shop,Steakhouse,South Indian Restaurant,Electronics Store,Boutique,Shopping Mall,Malay Restaurant
9,Kings Cross Station,Café,Italian Restaurant,Coffee Shop,Pub,Bar,Pizza Place,Sushi Restaurant,Thai Restaurant,Japanese Restaurant,Lounge,Indian Restaurant,Dumpling Restaurant,Wine Bar,Speakeasy,Burger Joint,Australian Restaurant,Hotel,Mexican Restaurant,Sandwich Place,Gym
14,Chatswood Station,Café,Coffee Shop,Food Court,Thai Restaurant,Japanese Restaurant,Gym,Chinese Restaurant,Ramen Restaurant,Chaat Place,Portuguese Restaurant,Burger Joint,Bubble Tea Shop,Tea Room,Shopping Mall,Dumpling Restaurant,Fried Chicken Joint,Performing Arts Venue,Market,Malay Restaurant,Sushi Restaurant
18,Milsons Point Station,Café,Park,Theme Park Ride / Attraction,Italian Restaurant,Thai Restaurant,Bar,Train Station,Bakery,Lounge,Food & Drink Shop,Flea Market,Fish Market,Asian Restaurant,Seafood Restaurant,Scenic Lookout,Burger Joint,Pub,Pool,Pizza Place,Pier
20,Artarmon Station,Café,Park,Japanese Restaurant,Thai Restaurant,Sandwich Place,Furniture / Home Store,Sushi Restaurant,BBQ Joint,Asian Restaurant,Motel,Diner,Ramen Restaurant,Convenience Store,Falafel Restaurant,Food Court,Food & Drink Shop,Flea Market,Fish Market,Dim Sum Restaurant,Fast Food Restaurant
21,Rhodes Station,Café,Dim Sum Restaurant,Japanese Restaurant,Fast Food Restaurant,Australian Restaurant,Asian Restaurant,Mexican Restaurant,Gym,Shopping Mall,Sandwich Place,Big Box Store,Chinese Restaurant,Chocolate Shop,Food Court,Movie Theater,Korean Restaurant,Park,Italian Restaurant,Trail,American Restaurant
28,St Peters Station,Café,Pub,Thai Restaurant,Gym / Fitness Center,Theater,Park,Garden Center,Greek Restaurant,Grocery Store,Indian Restaurant,Italian Restaurant,Pizza Place,Clothing Store,Egyptian Restaurant,Event Space,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market
33,Redfern Station,Café,Bar,Pub,Pizza Place,Bakery,Thai Restaurant,Coffee Shop,Japanese Restaurant,Grocery Store,Italian Restaurant,Ice Cream Shop,Bus Station,Gymnastics Gym,Restaurant,Chinese Restaurant,Gym / Fitness Center,Ramen Restaurant,Cocktail Bar,Market,Platform
37,Mascot Station,Café,Hotel,Coffee Shop,Chinese Restaurant,Supermarket,Thai Restaurant,Pizza Place,Park,Bus Stop,Ramen Restaurant,Bakery,Australian Restaurant,Sandwich Place,Platform,Sports Bar,Gym,Japanese Restaurant,Convenience Store,Wine Shop,Cosmetics Shop
40,Petersham Station,Café,Portuguese Restaurant,Pool,Pizza Place,Bar,Grocery Store,Liquor Store,Falafel Restaurant,Park,Pub,Coffee Shop,Chinese Restaurant,Bowling Green,Beer Garden,Yoga Studio,Athletics & Sports,Australian Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant


### 5.3 Cluster 3

In [27]:
station_merged.loc[station_merged['Cluster Labels'] == 2, station_merged.columns[[0] + list(range(4, station_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,Town Hall Station,Café,Japanese Restaurant,Hotel,Coffee Shop,Bookstore,Cocktail Bar,Korean Restaurant,Gym,Thai Restaurant,Shopping Mall,Burger Joint,Record Shop,Speakeasy,Bar,Australian Restaurant,Tea Room,Hobby Shop,Hotel Bar,Ice Cream Shop,Malay Restaurant
2,Hurstville Station,Chinese Restaurant,Fast Food Restaurant,Supermarket,Café,Bakery,Dumpling Restaurant,Vietnamese Restaurant,Coffee Shop,Japanese Restaurant,Bubble Tea Shop,Men's Store,Middle Eastern Restaurant,Souvlaki Shop,Accessories Store,Shopping Mall,Sandwich Place,Clothing Store,Electronics Store,Department Store,Bar
3,Ashfield Station,Dumpling Restaurant,Asian Restaurant,Shanghai Restaurant,Electronics Store,Japanese Restaurant,Supermarket,Chinese Restaurant,Coffee Shop,Korean Restaurant,Falafel Restaurant,Café,Department Store,Restaurant,Malay Restaurant,Bar,Liquor Store,Thai Restaurant,Polish Restaurant,Platform,Pharmacy
7,Bankstown Station,Vietnamese Restaurant,Café,Buffet,Coffee Shop,Grocery Store,Middle Eastern Restaurant,Steakhouse,Chinese Restaurant,Sports Bar,Department Store,Sports Club,Bar,Supermarket,Record Shop,Portuguese Restaurant,Multiplex,Shopping Mall,Bus Station,Fast Food Restaurant,Sandwich Place
8,Auburn Station,Café,Grocery Store,Bakery,Turkish Restaurant,Supermarket,Tea Room,Thai Restaurant,Pakistani Restaurant,Lebanese Restaurant,Gym,Department Store,Fast Food Restaurant,Portuguese Restaurant,Persian Restaurant,Dessert Shop,Afghan Restaurant,Kebab Restaurant,Farmers Market,Falafel Restaurant,Event Space
10,International Airport Station,Airport Lounge,Coffee Shop,Café,Juice Bar,Bakery,Electronics Store,Fast Food Restaurant,Seafood Restaurant,Scenic Lookout,Lingerie Store,Thai Restaurant,Australian Restaurant,Mobile Phone Shop,Duty-free Shop,Train Station,Hotel,Airport Terminal,Pizza Place,Vietnamese Restaurant,Breakfast Spot
11,Wolli Creek Station,Park,Gym / Fitness Center,Athletics & Sports,Train Station,Bakery,Street Food Gathering,Liquor Store,Chinese Restaurant,Asian Restaurant,Café,Dumpling Restaurant,Supermarket,Sushi Restaurant,Coffee Shop,Platform,Pizza Place,Farmers Market,Fast Food Restaurant,Event Space,Fish Market
12,Kogarah Station,Café,Thai Restaurant,Coffee Shop,Pub,Platform,Supermarket,Lebanese Restaurant,Portuguese Restaurant,Train Station,Fast Food Restaurant,Chinese Restaurant,Japanese Restaurant,Italian Restaurant,Shopping Mall,Pizza Place,Convenience Store,Bakery,Sandwich Place,Event Space,Electronics Store
13,Campsie Station,Malay Restaurant,Playground,Korean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Shopping Mall,Sushi Restaurant,Café,Liquor Store,Food & Drink Shop,Gym,Chinese Restaurant,Pharmacy,Department Store,Japanese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Fish Market,Dessert Shop,Farmers Market
17,Parramatta Station,Café,Asian Restaurant,Chinese Restaurant,Department Store,Coffee Shop,Pub,Fast Food Restaurant,Sandwich Place,Sushi Restaurant,Indian Restaurant,Thai Restaurant,Burger Joint,Bakery,Mexican Restaurant,Supermarket,Juice Bar,Gym / Fitness Center,Multiplex,Vietnamese Restaurant,Malay Restaurant


### 5.4 Cluster 4

In [28]:
station_merged.loc[station_merged['Cluster Labels'] == 3, station_merged.columns[[0] + list(range(4, station_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
16,Strathfield Station,Korean Restaurant,Café,Japanese Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Platform,Thai Restaurant,Malay Restaurant,Liquor Store,Supermarket,Chinese Restaurant,Plaza,Fast Food Restaurant,Sports Bar,Bookstore,Bus Station,Burger Joint,Bakery,BBQ Joint,Shopping Mall


### 5.5 Cluster 5

In [29]:
station_merged.loc[station_merged['Cluster Labels'] == 4, station_merged.columns[[0] + list(range(4, station_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
25,Green Square Station,Café,Sporting Goods Shop,Coffee Shop,Furniture / Home Store,Supermarket,Electronics Store,Pet Store,Pub,Thai Restaurant,Bar,Gym,Train Station,Farmers Market,Yoga Studio,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Falafel Restaurant,Egyptian Restaurant
30,Sydenham Station,Café,Brewery,Gym,Speakeasy,Coffee Shop,Fish Market,Soccer Field,Event Space,Food Court,Food & Drink Shop,Flea Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Yoga Studio,French Restaurant,Electronics Store,Egyptian Restaurant,Duty-free Shop,Dumpling Restaurant
32,St Leonards Station,Café,Coffee Shop,Chinese Restaurant,Vietnamese Restaurant,Gym,Office,Sandwich Place,Sushi Restaurant,Bakery,Thai Restaurant,Restaurant,Noodle House,Asian Restaurant,Middle Eastern Restaurant,Pub,Playground,Convenience Store,Pizza Place,Souvlaki Shop,Discount Store
